In [1]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import torch

/Users/vivekvajipey/miniconda3/envs/reasoning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/vivekvajipey/miniconda3/envs/reasoning/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2").to(device)

Default Cosine Similarity Method

In [ ]:
train_examples = [
    InputExample(texts=[
        "Step 2: Convert the length of the newest movie from hours to minutes. Since there are 60 minutes in an hour, we can calculate the length of the newest movie in minutes by multiplying the length in hours by 60. 3.2 hours * 60 minutes/hour = 192 minutes", 
        "Step 2: Convert the bake time of the larger cake from hours to minutes. There are 60 minutes in an hour, so we calculate the bake time of the larger cake in minutes by multiplying the bake time in hours by 60. 3.2 hours * 60 minutes/hour = 192 minutes."
        ], 
        label=1.0),
    InputExample(texts=[
        "Step 2: Convert the length of the newest movie from hours to minutes. Since there are 60 minutes in an hour, we can calculate the length of the newest movie in minutes by multiplying the length in hours by 60. 3.2 hours * 60 minutes/hour = 192 minutes", 
        "To convert this back to inches, we multiply by 12: 4.16 feet * 12 inches/foot = 49.92 inches"
        ], 
        label=0.0),
]

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Contrastive Loss Method

In [2]:
import pandas as pd
df = pd.read_csv('data/gsm8k_cl_trans_para1_112_autosplit.csv')
train_df = df.loc[df.split == 'train']
val_df = df.loc[df.split == 'val']

In [6]:
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
# from sentence_transformers_evaluator import LossEvaluator


device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

train_examples = []
for idx, row in train_df[:5].iterrows():
    train_examples.append(InputExample(texts=[row['Reasoning Trace'], row['Positive'][0]], label=1))
    for neg_ex in row['Negative']:
        train_examples.append(InputExample(texts=[row['Reasoning Trace'], neg_ex], label=0))

train_batch_size = 8
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.ContrastiveLoss(model=model)

val_examples = []
for idx, row in val_df[:3].iterrows():
    val_examples.append(InputExample(texts=[row['Reasoning Trace'], row['Positive'][0]], label=1))
    for neg_ex in row['Negative']:
        val_examples.append(InputExample(texts=[row['Reasoning Trace'], neg_ex], label=0))

val_loader = DataLoader(val_examples, batch_size=64)
# val_evaluator = LossEvaluator(val_loader, loss_model=train_loss, log_dir='logs/', name='all-MiniLM-L6-v2-val_trial0')
val_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    val_examples, batch_size=train_batch_size, name='all-MiniLM-L6-v2-val_trial0'
)

In [8]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=1, evaluator=val_evaluator, evaluation_steps=10000)

Epoch: 100%|██████████| 1/1 [01:41<00:00, 101.84s/it]
